<a href="https://colab.research.google.com/github/argennof/M_Predi_Lluvias_Extremas_10/blob/main/TP2_Balbuena_Serrantes_Fonnegra_Velazquez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mentoría "Predicción de lluvias extremas"

- Grupo 1:
  - Florencia Balbuena
  - Sebastián Serantes
  - Diana Fonnegra
  - Laura Velazquez

# TP 2: Análisis y Curación de Datos

In [1]:
import pandas as pd
# import funciones 
# from funciones import get_hydro_year

Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas/master/dataset/datos_diarios_cordoba.csv",\
                    parse_dates = True, index_col="fecha")
id_Datos = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas\
/master/dataset/metadatos_estaciones.csv")

## CONSIGNA TP2:

En este TP, nos enfocaremos en la cuenca del río Carcarañá. Por lo tanto, las estaciones que utilizaremos del dataset anterior son las siguientes:
- Pilar
- Rio Cuarto
- Marcos Juarez
- La Florida
- Pueblo Andino

En todos los casos, considerar el año hidrológico como del 1/agosto al 31/julio.


## 1. Análisis y curación de datos de una API - formato JSON

Referencias: https://darksky.net/dev/docs

  1. Extraer los datos para las coordenadas de las estaciones antes indicadas.
  2. ¿Cuántos años de datos hay?
  3. ¿Existen valores faltantes? Cuantificarlos.
  3. Generar un nuevo dataset a partir de los datos (time, summary, precipAccumulation, temperature, humidity, pressure, windSpeed, uvIndex, temperatureMin, temperatureMax.
  4. Hacer las conversiones de unidades necesarias a estos datos para poder compararlos con el dataset "datos_diarios_cordoba.csv".
  5. Comparar los estadísticos de estos datasets con los de cada estación.


## 2. Análisis y curación de datos de un Excel y un TXT

Referencias TerraClimate: http://www.climatologylab.org/terraclimate.html

  1. Leer los datos de "TerraClimate.csv" y determinar para cada variable los valores promedios anuales (correspondiente a cada año hidrológico).
  2. Hacer las conversiones de unidades necesarias para poder compararlos con cada estación.
  3. Determinar para las estaciones de la cuenca del Carcarañá, los valores promedios anuales (año hidrológico) y comparar los estadísticos de este dataset con los de "datos_diarios_cordoba.csv".
  4. Utilizar el dataset de TerraClimate para rellenar los datos faltantes en "datos_diarios_cordoba.csv" y guardar como un dataset nuevo. Hacer esto para todas las variables salvo para lluvia.
  5. Agregar las demás variables del dataset de TerraClimate al creado en el punto 4. 


## 3. Análisis y curación de datos de un Excel y un TXT

Referencias CHIRPS: https://disasters.nasa.gov/instruments/imerg

  1. En cada año hidrológico, leer los datos de "CHIRPS.txt" y determinar la Precipitaciones Máximas Diarias Anuales (PMDA) y las Precipitaciones Totales Anuales para cada estación.
  2. Graficar y comparar los estadísticos de estos datasets con los de cada estación (dataset "datos_diarios_cordoba.csv").
  3. ¿Es posible rellenar la serie del dataset "datos_diarios_cordoba.csv"?
  4. Agregar los nuevos features al dataset creado en el punto 5 de la parte 2 y exportar el dataset nuevo en formato csv.
  5. A partir del dataset del punto 4, elaborar matrices de correlación entre las diferentes variables y las series de Precipitaciones Máximas Diarias Anuales (PMDA) para cada estación.


## 4. Análisis y curación de Índices Climáticos en Excel

Referencias: https://psl.noaa.gov/data/climateindices/list/

  1. Leer el archivo "indices_clim.xlsx" y evaluar la cantidad de valores faltantes.
  2. Convertir los datos faltantes en NaN.
  3. Hacer matrices de correlación entre los diferentes índices climáticos y las series de Precipitaciones Máximas Diarias Anuales (PMDA) en las diferentes estaciones. Hacer esto para cada año hidrológico.
  4. Repetir lo mismo para la serie de caudales máximos diarios anuales (QMDA) en Pueblo Andino.

### Rta//
### Parte 1:

In [ ]:
def get_hydro_year(date_, start_month=7):

  '''
  Función que define el año hidrológico
  en el caso del proyecto este se define entre el -> 1/agosto al 31/julio
  '''

  year = date_.strftime('%Y')
  month = date_.strftime('%m')
  offset = 1 if int(month) >= start_month - 1 else 0
  hydro_year = int(year) + offset - 1
  return hydro_year


  1. Extraer los datos para las coordenadas de las estaciones:
 - Pilar
- Rio Cuarto
- Marcos Juarez
- La Florida
- Pueblo Andino

- En todos los casos, considerar el año hidrológico como del 1/agosto al 31/julio.







#### 1 a) Importamos algunas funcionalidades y los datos:

In [ ]:
import pandas as pd
import datetime
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.25)


import requests
import pandas as pd
from collections import namedtuple

#### * Filtramos la información de interes a partir del conjunto de datos disponible en [Git](https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas/master/dataset/metadatos_estaciones.csv). A su vez consideramos un grupo de estaciones:

In [ ]:
ruta_git='https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas/master/dataset/'

# Metadatos:
id_Datos=pd.read_csv(ruta_git + "metadatos_estaciones.csv", )
stations_dataset=id_Datos.copy()

# Convertimos el formato string de fecha a formato date-time pandas
id_Datos['fecha_inicio'] = pd.to_datetime(id_Datos.fecha_inicio)
id_Datos['Date_Month'] = pd.to_datetime(id_Datos.fecha_inicio.dt.strftime('%Y - %m'))

#Estaciones de interés:
zonas_int = ['Pilar Obs.','Rio Cuarto Aero','Marcos Juarez Aero','La Florida','Pueblo Andino']

# filtramos estas estaciones:
filtered_df = stations_dataset.loc[stations_dataset.nombre.isin(zonas_int)]


###   1. Extraer los datos para las coordenadas de las estaciones antes indicadas. 
###   2. ¿Cuántos años de datos hay?
Rta//:

### * Nota: la página presenta restricciones en cuanto a sus consultas (1000 por día, es por esta razon que se dejan indicadas las consultas de estacion puntual e iterando sobre el data frame: `filtered_df`)...
- Estaciones de manera individual:
  * Iniciando consulta desde: Thu Jul 01 2021 05:15:48 GMT+0000 - Unix: 1625116548.
  * Hasta:  Wed Jun 22 2022 21:21:00 GMT+0000 - Unix: 1655932860.
  * [Variables](https://www.home-assistant.io/integrations/darksky/#precip_probability) seleccionadas / Unidades SI.
      - time: horario en tiempo Unix.
      - summary: descripción. Formato: String.
      - precipAccumulation: precipitación en milímetros por día [mm] 
      - precipIntensity: precipitación en milímetros por hora [mm]
      - precipIntensityMax: precipitación máxima por hora en milímetros [mm]
      - precipProbability: probabilidad de precipitación.
      - precipType: tipo de precipitación que ocurre en el momento dado
      - humidity: humedad atmosférica.
      - pressure: presión barométrica.
      - windSpeed: velocidad del viento en kilometros por hora [km/h]
      - uvIndex: índice UV.
      - temperatureMin: temperatura mínima en Celsius [°C]
      - temperatureMax:temperatura máxima en Celsius [°C]
      - omm_esta: nombre de la estación
      - codigo: codificación OMM.

 

In [ ]:
import requests
import pandas as pd
from collections import namedtuple


# Listado de variables:
var = 'time,summary,precipAccumulation,precipIntensity,precipIntensityMaxTime,\
precipProbability,precipType,humidity,pressure,windSpeed,uvIndex,temperatureMin,temperatureMax,omm_esta,codigo'

# Transformo en listado
features = var.split(',')
# print(features) #Control de variables

DailySummary = namedtuple("DailySummary", features)
DailySummary

df1 = pd.DataFrame()
key = '9349598e9f11f5eddceb6791daa6d787'
baseUrl="https://api.darksky.net/forecast"

df2 = pd.DataFrame()
lista_consulta=[]

# Caracteristicas extraidas de metadato: 
lon_dec=-64.276973
lat_dec=-33.0955
omm_esta = "Rio Cuarto Aero"
codigo = 87453

print(f'\nUrl para la estación: {omm_esta} - código: {codigo}')
 
for unix in range(1625116548, 1655932860, 86400):
  consulta = f'{baseUrl}/{key}/{lat_dec},{lon_dec},{unix}?exclude=flags,hourly&units=ca'

  # print(consulta)
  try:
    r = requests.get(consulta)
    data = r.json()
    df = pd.DataFrame(list(data["daily"]["data"]))
    df['omm_esta'] = omm_esta
    df['codigo'] = codigo
    df2 = pd.concat([df2, df])
    lista_consulta.append(data)
  
  except:
    # En caso que la consulta falle...
    print(f'La estación: {omm_esta} - No posee datos para la fecha en formato unix: {unix}')
    pass
  # break


#### * Primer resultado de la consulta - JSON:

In [ ]:
print(f'Resultado de las consultas \n {lista_consulta[0]}')


#### * Las transformamos en DataFrame: `res` y vemos el primer registro del mismo.

In [ ]:
res = pd.DataFrame(df2, columns=features)
res.reset_index(drop=True)
res.head(1)


  ### 3. ¿Existen valores faltantes? Cuantificarlos.
  - Las variables que presentan datos faltantes son:


In [ ]:
print(res.info())

# al parecer en la descarga no se presentaban datos faltantes:
res['precipIntensity'].isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   time                    0 non-null      float64
 1   summary                 0 non-null      float64
 2   precipAccumulation      0 non-null      float64
 3   precipIntensity         0 non-null      float64
 4   precipIntensityMaxTime  0 non-null      float64
 5   precipProbability       0 non-null      float64
 6   precipType              0 non-null      float64
 7   humidity                0 non-null      float64
 8   pressure                0 non-null      float64
 9   windSpeed               0 non-null      float64
 10  uvIndex                 0 non-null      float64
 11  temperatureMin          0 non-null      float64
 12  temperatureMax          0 non-null      float64
 13  omm_esta                0 non-null      float64
 14  codigo                  0 non-null      float64
dtypes: flo

0

#### * Calculamos la precipitación acumulada diaria y transformamos el tiempo Unix escalas convecionales: 
- precipAccumulation = precipIntensity * 24 hr obtenida de [Foro](https://stackoverflow.com/questions/55866558/should-darksky-time-machine-historical-data-show-precipaccumulation-when-preci)

In [ ]:
from datetime import datetime

# transformando de tiempo Unix a unidades convencionales:
res['Date'] = pd.to_datetime(res["time"], unit = 's').dt.strftime("%Y-%m-%d")

# Calculamos la precipitación acumulada:
res['precipAccumulation']= res['precipIntensity']*24 # 24 hrs
res['temperatura'] = (res['temperatureMax'] + res['temperatureMin']) * 0.5 

In [ ]:
res.info() 



<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   time                    0 non-null      float64
 1   summary                 0 non-null      float64
 2   precipAccumulation      0 non-null      float64
 3   precipIntensity         0 non-null      float64
 4   precipIntensityMaxTime  0 non-null      float64
 5   precipProbability       0 non-null      float64
 6   precipType              0 non-null      float64
 7   humidity                0 non-null      float64
 8   pressure                0 non-null      float64
 9   windSpeed               0 non-null      float64
 10  uvIndex                 0 non-null      float64
 11  temperatureMin          0 non-null      float64
 12  temperatureMax          0 non-null      float64
 13  omm_esta                0 non-null      float64
 14  codigo                  0 non-null      float64
 15  Date  

3. Generar un nuevo dataset a partir de los datos (time, summary, precipAccumulation, temperature, humidity, pressure, windSpeed, uvIndex, temperatureMin, temperatureMax.

In [ ]:
df_riocuarto = res.loc[:,['Date','summary','temperatura', 'humidity', 'pressure', \
                          'windSpeed', 'uvIndex', 'temperatureMin', 'temperatureMax']].copy()

In [ ]:
import missingno as msno
msno.matrix(df_riocuarto,figsize=(20, 6), fontsize=12, color=[0,0,0.2])

  * Almacenando los datos en formato CSV:

In [ ]:
df_riocuarto.to_csv('87453_rio_cuarto.csv', sep=';',index=False)
from google.colab import files
files.download('87453_rio_cuarto.csv')

* Almacenando los datos en formato .TXT:

In [ ]:
listitems = lista_consulta.copy()
with open('consulta_87453_r4.txt', 'w') as temp_file:
    for item in listitems:
        temp_file.write("%s\n" % item)
file = open('consulta_87453_r4.txt', 'r')
#print(file.read()])
print(listitems[0])

### * Se realizo una prueba iterando a todas las estaciones [Script - todas las estaciones](https://github.com/argennof/M_Predi_Lluvias_Extremas_10/blob/main/todas_estaciones_consulta.py). 

In [ ]:
from google.colab import files
files.download('consulta_87453_r4.txt')

#### 4. Hacer las conversiones de unidades necesarias a estos datos para poder compararlos con el dataset "datos_diarios_cordoba.csv".



In [ ]:
df_riocuarto = df_rc.copy()
# df_riocuarto['Date'] = pd.to_datetime(df_riocuarto['Date'])
# df_riocuarto['Date'] = df_riocuarto['Date'].mask(df_riocuarto['Date'].dt.year > 2019, 
#                                         df_riocuarto['Date'] - pd.offsets.DateOffset(years=100))
# df_riocuarto['hydro_year'] = df_riocuarto['Date'].apply(lambda Date: get_hydro_year(Date))
# df_riocuarto

In [ ]:
not_var_cols = ['Date', 'id', 'omm_id', 'nombre', 'hydro_year']
var_cols = [c for c in df_rc.columns if c not in not_var_cols]
df_rc_var = df_rc[var_cols]
df_rc_var_mean = pd.DataFrame()

for col in var_cols:
  df_rc_var_mean[col] = df_rc.groupby(['omm_id','hydro_year'])[col].mean()

df_rc_var_mean


## 2. Análisis y curación de datos de un Excel y un TXT

1. Leer los datos de "TerraClimate.csv" y determinar para cada variable los valores promedios anuales (correspondiente a cada año hidrológico).


#### * Leemos los datos del archivo "TerraClimate.csv" y los guardamos en nuestro dataset.

In [ ]:
datos_tc = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas/main/dataset/TerraClimate.csv")
datos_tc

,Date,Lluvia_mm,ETreal_mm,HumedadSuelo_mm,RadSolar_Wpm2,Temp_Min,Temp_Max,PresionVapor_hPa,VelViento_mps,IndicePalmer_porc,id,omm_id,nombre
0,1-Jan-58,165.0,1570.0,32.0,281.3,15.4,29.8,19.59,3.27,0.78,5,87349,Pilar Obs.
1,1-Feb-58,75.0,713.0,30.0,246.3,15.8,29.5,20.88,3.08,-0.45,5,87349,Pilar Obs.
2,1-Mar-58,119.0,1123.0,33.0,213.5,13.0,26.1,17.72,3.31,0.23,5,87349,Pilar Obs.
3,1-Apr-58,33.0,313.0,31.0,151.6,10.7,23.3,15.13,2.65,-0.46,5,87349,Pilar Obs.
4,1-May-58,13.0,119.0,28.0,109.8,6.1,19.8,10.86,2.91,-0.76,5,87349,Pilar Obs.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,1-Aug-19,2.0,19.0,38.0,173.9,1.5,16.2,4.24,5.15,-2.08,13,1,La Florida
2972,1-Sep-19,9.0,85.0,36.0,228.0,4.2,19.1,4.79,5.50,-2.74,13,1,La Florida
2973,1-Oct-19,22.0,209.0,34.0,222.7,7.1,20.8,7.46,5.97,-3.75,13,1,La Florida
2974,1-Nov-19,66.0,626.0,33.0,271.1,11.5,25.8,12.12,5.77,-3.96,13,1,La Florida


#### * Asignamos el año hidrologico correspondinte a cada registro.

In [ ]:
datos_tc['Date'] = pd.to_datetime(datos_tc['Date'])
datos_tc['Date'] = datos_tc['Date'].mask(datos_tc['Date'].dt.year > 2019, 
                                        datos_tc['Date'] - pd.offsets.DateOffset(years=100))
datos_tc['hydro_year'] = datos_tc['Date'].apply(lambda Date: get_hydro_year(Date))
datos_tc

,Date,Lluvia_mm,ETreal_mm,HumedadSuelo_mm,RadSolar_Wpm2,Temp_Min,Temp_Max,PresionVapor_hPa,VelViento_mps,IndicePalmer_porc,id,omm_id,nombre,hydro_year
0,1958-01-01,165.0,1570.0,32.0,281.3,15.4,29.8,19.59,3.27,0.78,5,87349,Pilar Obs.,1957
1,1958-02-01,75.0,713.0,30.0,246.3,15.8,29.5,20.88,3.08,-0.45,5,87349,Pilar Obs.,1957
2,1958-03-01,119.0,1123.0,33.0,213.5,13.0,26.1,17.72,3.31,0.23,5,87349,Pilar Obs.,1957
3,1958-04-01,33.0,313.0,31.0,151.6,10.7,23.3,15.13,2.65,-0.46,5,87349,Pilar Obs.,1957
4,1958-05-01,13.0,119.0,28.0,109.8,6.1,19.8,10.86,2.91,-0.76,5,87349,Pilar Obs.,1957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,2019-08-01,2.0,19.0,38.0,173.9,1.5,16.2,4.24,5.15,-2.08,13,1,La Florida,2019
2972,2019-09-01,9.0,85.0,36.0,228.0,4.2,19.1,4.79,5.50,-2.74,13,1,La Florida,2019
2973,2019-10-01,22.0,209.0,34.0,222.7,7.1,20.8,7.46,5.97,-3.75,13,1,La Florida,2019
2974,2019-11-01,66.0,626.0,33.0,271.1,11.5,25.8,12.12,5.77,-3.96,13,1,La Florida,2019


#### * Calculamos la media anual (año hidrologico) de cada una de las variables para cada una de las estaciones de interes.

In [ ]:
not_var_cols = ['Date', 'id', 'omm_id', 'nombre', 'hydro_year']
var_cols = [c for c in datos_tc.columns if c not in not_var_cols]
datos_tc_var = datos_tc[var_cols]
datos_tc_var_mean = pd.DataFrame()

for col in var_cols:
  datos_tc_var_mean[col] = datos_tc.groupby(['omm_id','hydro_year'])[col].mean()

datos_tc_var_mean


Lluvia_mm   ETreal_mm  HumedadSuelo_mm  RadSolar_Wpm2  \
omm_id hydro_year                                                          
1      1957        66.600000  632.800000        45.400000     200.960000   
       1958        59.416667  564.000000        30.666667     189.958333   
       1959        54.833333  519.583333        20.666667     191.216667   
       1960        69.250000  643.583333        53.916667     193.958333   
       1961        55.750000  533.916667        84.416667     195.908333   
...                      ...         ...              ...            ...   
87466  2015        85.833333  722.666667       165.250000     188.975000   
       2016        83.416667  753.666667       209.750000     200.050000   
       2017        70.916667  636.083333       185.000000     206.616667   
       2018        75.916667  753.583333       158.416667     198.408333   
       2019        31.857143  301.714286        47.000000     196.514286   

                    Temp_Min   Temp_Max  PresionVapor_hPa  VelViento_mps  \
omm_id hydro_year                                                          
1      1957         8.360000  22.320000         11.452000       3.946000   
       1958         5.866667  19.916667          8.875833       4.153333   
       1959         5.316667  21.208333          9.230833       4.045833   
       1960         5.975000  20.766667          9.322500       4.111667   
       1961         6.191667  20.650000          9.508333       4.167500   
...                      ...        ...               ...            ...   
87466  2015        10.750000  23.783333         14.811667       3.440000   
       2016        10.500000  23.650000         14.382500       3.509167   
       2017        11.133333  24.625000         14.662500       3.678333   
       2018        10.125000  23.450000         14.905000       3.711667   
       2019         9.414286  22.585714         11.555714       3.954286   

                   IndicePalmer_porc  
omm_id hydro_year                     
1      1957                -0.054000  
       1958                 0.305833  
       1959                 1.140833  
       1960                 0.762500  
       1961                 1.642500  
...                              ...  
87466  2015                 0.444167  
       2016                 2.520000  
       2017                -0.045000  
       2018                 0.440833  
       2019                -2.568571  

[252 rows x 9 columns]

2. Hacer las conversiones de unidades necesarias para poder compararlos con cada estación.

#### Rta: no es necesario realizar conversiones.

3. Determinar para las estaciones de la cuenca del Carcarañá, los valores promedios anuales (año hidrológico) y comparar los estadísticos de este dataset con los de "datos_diarios_cordoba.csv".

In [ ]:
datos_dcor = pd.read_csv("https://raw.githubusercontent.com/Rondamon/2022_Mentoria_LluviasExtremas/main/dataset/datos_diarios_cordoba.csv")
datos_dcor['fecha'] = pd.to_datetime(datos_dcor['fecha'])
datos_dcor['fecha'] = datos_dcor['fecha'].mask(datos_dcor['fecha'].dt.year > 2019, 
                                        datos_dcor['fecha'] - pd.offsets.DateOffset(years=100))
datos_dcor['hydro_year'] = datos_dcor['fecha'].apply(lambda Date: get_hydro_year(Date))
datos_dcor

,id,omm_id,fecha,helio,hr,nub,prcp,pres_est,pres_nm,td,tmax,tmed,tmin,vmax_d,vmax_f,vmed,caudal,hydro_year
0,1,87244,1961-01-01,NaN,62.0,0.0,0.0,965.9,NaN,15.9,32.3,25.1,13.5,NaN,NaN,1.0,NaN,1960
1,2,87244,1961-01-02,NaN,66.0,6.0,28.0,969.9,NaN,15.5,27.5,22.2,18.3,NaN,NaN,1.0,NaN,1960
2,3,87244,1961-01-03,NaN,92.0,8.0,15.4,972.6,NaN,13.1,21.7,14.4,12.9,NaN,NaN,4.5,NaN,1960
3,4,87244,1961-01-04,NaN,84.0,5.0,0.0,969.6,NaN,17.4,23.1,20.4,13.9,NaN,NaN,0.3,NaN,1960
4,5,87244,1961-01-05,NaN,84.0,5.0,3.6,965.6,NaN,17.8,23.9,20.7,15.5,NaN,NaN,1.5,NaN,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249622,249623,2,2017-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.50303,2017
249623,249624,2,2017-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.93904,2017
249624,249625,2,2017-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.33246,2017
249625,249626,2,2017-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.72549,2017


## 4. Análisis y curación de Índices Climáticos en Excel

Referencias: https://psl.noaa.gov/data/climateindices/list/

  1. Leer el archivo "indices_clim.xlsx" y evaluar la cantidad de valores faltantes.
  2. Convertir los datos faltantes en NaN.
  3. Hacer matrices de correlación entre los diferentes índices climáticos y las series de Precipitaciones Máximas Diarias Anuales (PMDA) en las diferentes estaciones. Hacer esto para cada año hidrológico.
  4. Repetir lo mismo para la serie de caudales máximos diarios anuales (QMDA) en Pueblo Andino.

### 4.a) Cargando el conjunto de datos

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

url='https://github.com/Rondamon/2022_Mentoria_LluviasExtremas/blob/main/dataset/indices_clim.xlsx?raw=true'
with pd.ExcelFile(url) as xls:
  db_index=pd.read_excel(url,parse_dates = True, index_col='date')
  
# db_index.columns.to_list
db_index.head(3)

### * Evaluando la cantidad de variables así como sus valores faltantes:

In [ ]:
db_index.info()

### * Como se puede notar en la anterior contamos con 876 registros para cada una de las 43 posibles variables. A demás se puede resaltar que 5 variables: BEST, TNA, TSA, QBO, WHWP, AMO cuentan con 869 registros y son las más completas de la Base de Datos (BBDD). A su vez se destaca que en general el conjunto es de tipo flotante. 

### * Creamos una copia del dataframe `db_index` y lo denominamos `df`. Posteriormente, rellenamos cada uno de los valores faltantes con el argumento `np.nan`:

In [ ]:

df = db_index.copy()
df.fillna(np.nan, inplace=True)

### * Contabilizamos la cantidad de valores faltantes por variable (ordenados de menor a mayor):

In [ ]:
# Valores Faltantes:
missing_values_count = df.isna().sum()
missing_values_count.sort_values()

### * Calculamos el porcentaje de datos faltantes sobre el conjunto de datos:

In [ ]:
# how many total missing values do we have?
total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
print(f'porcentaje de faltantes en la base de datos: {((total_missing/total_cells) * 100):2f}')

### * Ahora lo caracterizamos visualmente:
- Matriz de datos:

In [ ]:
import missingno as msno
msno.matrix(df,figsize=(20, 6), fontsize=12, color=[0,0,0.2])

- Ordenados según la variable TSA:

In [ ]:
sorted = df.sort_values('TSA')
msno.matrix(sorted,figsize=(20, 6), fontsize=12, color=[0.5,0,0])

Revisar:

Tenemos que BEST, TNA, TSA, QBO, WHWP, AMO presentan 7 valores con NaN. Decidimos entonces que los imputaremos dado metodo `SimpleImputer` combinado con el valor que arroje `median()`:

In [ ]:
from sklearn.impute import SimpleImputer

values = ["BEST", "TNA", "TSA", "QBO","WHWP", "AMO"],#'PMM', "Trans-Nino Index TNI", "Nort Pacific Index NP"]

for i, value in enumerate(values):
  globals()[f'mediana_{values[i]}'] = db_index[values[i]].median()
  print(f'mediana_{values[i]}:')
  print(globals()[f'mediana_{values[i]}'])

  db_index[values[i]].fillna(globals()[f'mediana_{values[i]}'], inplace = True)

  globals()[f'missing_values_{values[i]}'] = db_index[values[i]].isna().sum()
  print(f'missing_values_{values[i]}:')
  print(globals()[f'missing_values_{values[i]}'])
  print("------ ------ ------ ------ ------ ------ ------ ")

 